In [1]:
!nvidia-smi

Thu Mar 11 17:36:45 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64       Driver Version: 440.64       CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  TITAN RTX           Off  | 00000000:03:00.0 Off |                  N/A |
| 41%   60C    P2    69W / 280W |  14012MiB / 24219MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  TITAN RTX           Off  | 00000000:04:00.0 Off |                  N/A |
| 41%   46C    P2    56W / 280W |   5923MiB / 24220MiB |      1%      Default |
+-------------------------------+----------------------+----------------------+
|   2  T

In [2]:
import torch
import random
import numpy as np
import matplotlib.pyplot as plt
import torchvision.datasets
from torchvision import transforms

In [3]:
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.backends.cudnn.deterministic = True

In [4]:
#также будем использовать аугментацию: рандомные повороты на 7 градусов и отражение относительно вертикали.

#преобразование трейна
train_tf = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(7),
    transforms.ToTensor()
])

#тест не меняем
test_tf = transforms.Compose([
    transforms.ToTensor()
])

train = torchvision.datasets.CIFAR10('./', download=True, train=True, transform=train_tf)
test = torchvision.datasets.CIFAR10('./', download=True, train=False, transform=test_tf)

Files already downloaded and verified
Files already downloaded and verified


In [5]:
#трейновая выборка
trainloader = torch.utils.data.DataLoader(dataset=train, batch_size=256)

#тестовая выборка
testloader = torch.utils.data.DataLoader(dataset=test, batch_size=256, shuffle=False)

In [6]:
import torch
device = torch.device('cuda:3' if torch.cuda.is_available() else 'cpu')

In [7]:
print(device)

cuda:3


In [8]:
import torchvision.models as models

net = models.resnet50(pretrained=True)
net = net.to(device)

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth


In [9]:
#В качестве лоса возмем кросс-энтропию. Оптимизатор - Адам
loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=1.0e-3)

#добавим уменьшение лернинг рейта, если выходим на плато. Это решение будем принимать по валидационной выборке.
lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, verbose=True, patience=7, factor=0.2)

In [10]:
def train(epoch):
    running_loss = 0.0
    train_acc = 0.0
    test_acc = 0.0
    len_train = 0.0
    len_test = 0.0
    
    loss_train = 0.0
  
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss_val = loss(outputs, targets)
        loss_val.backward()
        optimizer.step()
        accuracy_add = (outputs.argmax(dim=1) == targets).float().sum().data.cpu()
        train_acc += accuracy_add #не забываем считать метрику на тесте
        len_train += len(targets)
        loss_train += len(targets) * loss_val.item()
        running_loss += loss_val.item()
        
    for _, (inputs, targets) in enumerate(testloader):
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = net(inputs)
        accuracy_add = (outputs.argmax(dim=1) == targets).float().sum().data.cpu()
        test_acc += accuracy_add #не забываем считать метрику на тесте
        len_test += len(targets)
        
    lr_scheduler.step(running_loss)

    return train_acc / len_train, test_acc / len_test

In [11]:
#обучим нашу нейронную сеть, думаю, в районе 50-70 эпох будет достаточно. 
#Будем выводить на каждой эпохе accuracy на трейновой, валидационной и тестовой выборках

accuracy_history_test = []
accuracy_history_train = []

for epoch in range(0, 15):
    train_acc, test_acc = train(epoch)
    accuracy_history_test.append(test_acc)
    accuracy_history_train.append(train_acc)    
    print('Epoch:', epoch, '   acc_train:', np.round(train_acc.numpy(), 4), '   test_acc:', np.round(test_acc.numpy(), 4))

Epoch: 0    acc_train: 0.6551    test_acc: 0.7697
Epoch: 1    acc_train: 0.7962    test_acc: 0.8083
Epoch: 2    acc_train: 0.8333    test_acc: 0.8183
Epoch: 3    acc_train: 0.8524    test_acc: 0.8231
Epoch: 4    acc_train: 0.8691    test_acc: 0.83
Epoch: 5    acc_train: 0.8835    test_acc: 0.8384
Epoch: 6    acc_train: 0.8891    test_acc: 0.8399
Epoch: 7    acc_train: 0.8999    test_acc: 0.8367
Epoch: 8    acc_train: 0.9057    test_acc: 0.8398
Epoch: 9    acc_train: 0.9172    test_acc: 0.8412
Epoch: 10    acc_train: 0.9215    test_acc: 0.8386
Epoch: 11    acc_train: 0.9271    test_acc: 0.8416
Epoch: 12    acc_train: 0.9317    test_acc: 0.8407
Epoch: 13    acc_train: 0.9374    test_acc: 0.8418
Epoch: 14    acc_train: 0.9401    test_acc: 0.842


In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=[14, 5])
plt.plot(accuracy_history_test, label='accucacy on test set')
plt.plot(accuracy_history_train, label='accucacy on train set')
plt.legend(loc='best')
plt.title('Accuracy on train and test set on each epoch')
plt.grid('True')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')

In [12]:
net = net.eval()

In [13]:
torch.save(net, 'CIFAR10_ResNet50.pth')

In [ ]:
quit()